In [74]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import f_oneway

current_dir = os.getcwd()
project_root = os.path.dirname(os.path.dirname(current_dir))
PROCESSED_DATA_PATH = os.path.join(project_root, 'data', 'processed')
RAW_DATA_PATH = os.path.join(project_root, 'data', 'raw')
OUTPUT_PATH = os.path.join(project_root, 'output')
REPORTS_TABLES_PATH = os.path.join(project_root, "reports", "tables")

In [95]:
df = pd.read_csv(PROCESSED_DATA_PATH + "/dummy-efa.csv")

In [96]:
# Calculate the mean of performance (based on columns starting with "EPERFX")
eperfx_columns = [col for col in df.columns if col.startswith("EPERFX")]

# Check for non-numeric values
for col in eperfx_columns:
    if not pd.api.types.is_numeric_dtype(df[col]):
        print(f"Column {col} contains non-numeric data.")

# Check for NaN values
for col in eperfx_columns:
    if df[col].isna().any():
        print(f"Column {col} contains NaN values.")

df['Mean of Performance'] = df[eperfx_columns].mean(axis=1)

# Calculate the frequency of each category
category_counts = df['Category'].value_counts().reset_index()
category_counts.columns = ['Category', 'Frequency']

# Calculate the percentage of each category
total_rows = len(df)
category_counts['Percentage'] = (category_counts['Frequency'] / total_rows) * 100

# Aggregate data for each category
result_df = pd.merge(category_counts, df.groupby('Category')['Mean of Performance'].mean().reset_index(), on='Category')

# Rename columns for clarity
result_df.columns = ['Category', 'Frequency', 'Percentage', 'Mean of Performance']

# Sort by mean performance in descending order
result_df = result_df.sort_values(by='Mean of Performance', ascending=False)

# Reset the index
result_df = result_df.reset_index(drop=True)

# Print or work with 'result_df' as needed
print(result_df)


                   Category  Frequency  Percentage  Mean of Performance
0  High JIT & Environmental         67   38.285714             3.873134
1      Mainly Environmental         50   28.571429             3.680370
2                Mainly JIT         20   11.428571             3.616667
3   Low JIT & Environmental         38   21.714286             3.296784


In [97]:
# save results_df to csv
result_df.to_csv(PROCESSED_DATA_PATH + "/mean_perf_efa_suppliers.csv", index=False)

In [98]:
import pandas as pd

def dataframe_to_latex(df, file_name, label="tab:my_label", caption="My Caption"):
    """
    Convert a pandas DataFrame to a LaTeX table with specific column modifications,
    save it to a file, apply formatting to the numeric columns, and include label
    and caption for the table in LaTeX.

    Parameters:
    - df: pandas DataFrame to convert.
    - file_name: Name of the file to save the LaTeX code.
    - label: Label for the table in LaTeX.
    - caption: Caption for the table in LaTeX.
    """
    
    # Escape all LaTeX special characters in the DataFrame
    df = df.replace('&', '\\&', regex=True)
    
    # Format numbers in Python before converting to LaTeX
    df['Frequency'] = df['Frequency'].astype(int)
    df['Percentage'] = df['Percentage'].apply(lambda x: f"{x:.2f}")
    df['Mean of Performance'] = df['Mean of Performance'].apply(lambda x: f"{x:.2f}")

    # Create the LaTeX table code without the longtable format
    latex_code = df.to_latex(index=False, header=True, escape=False)

    # Wrap the tabular environment with the table float environment
    table_float = r'''\begin{table}[htbp]
    \centering
    \caption{''' + caption + r'''}
    \label{''' + label + r'''}
    ''' + latex_code + r'''
    \end{table}
    '''

    # Save to file
    with open(file_name, 'w') as file:
        file.write(table_float)


In [99]:
# # read csv
df = pd.read_csv(PROCESSED_DATA_PATH + "/mean_perf_efa.csv")

# latex_file_name = REPORTS_TABLES_PATH + "/performance_means_efa_suppliers.tex"
# dataframe_to_latex(df, latex_file_name, label="tab:your_label", caption="Practice Adoption (Environmental Practices 2 - Suppliers)")

In [100]:
# Step 1: Perform ANOVA to check for overall differences
anova_result = f_oneway(df[df['Category'] == 'High JIT & Environmental']['Mean of Performance'],
                        df[df['Category'] == 'Mainly Environmental']['Mean of Performance'],
                        df[df['Category'] == 'Mainly JIT']['Mean of Performance'],
                        df[df['Category'] == 'Low JIT & Environmental']['Mean of Performance'])

# Step 2: Check if ANOVA indicates significant differences
if anova_result.pvalue < 0.05:
    # Step 3: Perform Tukey's HSD test for pairwise comparisons
    tukey_results = pairwise_tukeyhsd(df['Mean of Performance'], df['Category'], alpha=0.05)

    # Display Tukey results
    print(tukey_results.summary())
else:
    print("No significant differences among categories based on ANOVA.")


No significant differences among categories based on ANOVA.


/Users/john.anderson/hpm/.venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4141: DegenerateDataWarning: all input arrays have length 1.  f_oneway requires that at least one input has length greater than 1.
  warnings.warn(stats.DegenerateDataWarning(msg))


In [66]:
import pandas as pd
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def tukey_to_latex(df, file_name, caption="Tukey Analysis", label="tab:your_label", alpha=0.05):
    """
    Perform ANOVA and Tukey HSD test on the given DataFrame, then convert the Tukey HSD test 
    results to a LaTeX table with a caption and label, and save it to a file.

    Parameters:
    - df: pandas DataFrame containing the data.
    - file_name: Name of the file to save the LaTeX code.
    - caption: Caption for the LaTeX table.
    - label: Label for the LaTeX table.
    - alpha: Significance level for Tukey HSD test.
    """
    # Perform ANOVA
    anova_result = f_oneway(
        df[df['Category'] == 'High JIT & Environmental']['Mean of Performance'],
        df[df['Category'] == 'Mainly Environmental']['Mean of Performance'],
        df[df['Category'] == 'Mainly JIT']['Mean of Performance'],
        df[df['Category'] == 'Low JIT & Environmental']['Mean of Performance']
    )
    
    # Initialize the LaTeX code for the table with caption and label
    latex_code = f"\\begin{{table}}[htbp]\n\\centering\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\begin{{tabular}}{{llrrrrr}}\n\\toprule\n"

    # Check if ANOVA is significant
    if anova_result.pvalue < alpha:
        # Perform Tukey HSD
        tukey_results = pairwise_tukeyhsd(df['Mean of Performance'], df['Category'], alpha=alpha)
        # Convert Tukey HSD results to a DataFrame
        tukey_df = pd.DataFrame(data=tukey_results._results_table.data[1:], columns=tukey_results._results_table.data[0])
        
        # Escape ampersands in group names and round all numeric columns to two decimal places
        tukey_df['group1'] = tukey_df['group1'].str.replace('&', '\\&', regex=False)
        tukey_df['group2'] = tukey_df['group2'].str.replace('&', '\\&', regex=False)
        numeric_cols = tukey_df.select_dtypes(include=['float64', 'float32']).columns
        tukey_df[numeric_cols] = tukey_df[numeric_cols].round(2)

        # Assuming 'tukey_df' is your DataFrame and you want to drop columns named 'col_to_drop1', 'col_to_drop2', etc.
        columns_to_drop = ['meandiff']
        tukey_df = tukey_df.drop(columns_to_drop, axis=1)
        
        # Convert DataFrame to LaTeX code, remove the tabular environment from the generated LaTeX table and append the content
        tukey_latex = tukey_df.to_latex(index=False, header=True, escape=False, formatters={
            # 'meandiff': '{:0.2f}'.format,
            'p-adj': '{:0.2f}'.format,
            'lower': '{:0.2f}'.format,
            'upper': '{:0.2f}'.format,
        })
        
        # Remove the tabular environment from the generated LaTeX table and append the content
        tukey_latex = tukey_latex.split("\n", 2)[-1].rsplit("\n", 1)[0]  # Remove the first and last lines containing tabular environment
        latex_code += tukey_latex
    else:
        latex_code += "No significant differences found by ANOVA.\n"

    # Correctly finish the LaTeX table setup
    latex_code += "\\end{table}"

    # Save to file
    with open(file_name, 'w') as file:
        file.write(latex_code)

# Example usage:
# Ensure that df is defined in your workspace before calling this function
# tukey_to_latex(df, 'tukey_results.tex')


In [45]:
tukey_to_latex(df, REPORTS_TABLES_PATH + '/tukey_results_efa_supp.tex', caption="Multiple Comparison of Means - Tukey HSD, FWER=0.05 (Environmental Practices 1 - General)", label="tab:your_label", alpha=0.05)